<a href="https://colab.research.google.com/github/AmiJoba22/RateMD-Datapipe/blob/main/pythonscripts%20/%20extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Get data from source to put in destination
#Destination has an address; source has an address
!pip install azure-storage-blob
!pip install pymongo

In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
# read the config file that is json
import json
with open('config.json') as config_file:
  config = json.load(config_file)

#read information about the destination
DESTINATION_CONNECTION_STRING = config['DESTINATION_CONNECTION_STRING']
DESTINATION_CONTAINER_NAME = config['DESTINATION_CONTAINER_NAME']
DESTINATION_FILE_NAME = config ['DESTINATION_FILE_NAME']

# read information about mongodb
MONGODB_CONNECTION_STRING = config ['MONGODB_CONNECTION_STRING']
MONGO_DB_NAME = config ['MONGO_DB_NAME']
MONGO_COLLECTION_NAME = config ['MONGO_COLLECTION_NAME']
print(MONGO_COLLECTION_NAME)

ratemd


In [3]:
# connect to mongodb and test the connection using ping
client = MongoClient(MONGODB_CONNECTION_STRING)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
db = client[MONGO_DB_NAME]
collection = db[MONGO_COLLECTION_NAME]
print(collection)

Pinged your deployment. You successfully connected to MongoDB!
Collection(Database(MongoClient(host=['20.232.135.212:27017'], document_class=dict, tz_aware=False, connect=True, authmechanism='DEFAULT', authsource='admin'), 'healthrate'), 'ratemd')


In [4]:
from azure.storage.blob import BlobServiceClient
import pandas as pd

CHUNK_SIZE = 500000
total_docs = collection.count_documents({})
print(f"Total documents in collection: {total_docs}")

blob_service_client = BlobServiceClient.from_connection_string(DESTINATION_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(DESTINATION_CONTAINER_NAME)

for skip in range(0, total_docs, CHUNK_SIZE):
    chunk_count = (skip // CHUNK_SIZE) + 1
    blob_chunk_name = f"{DESTINATION_FILE_NAME}_chunk_{chunk_count}.csv"

    # 🛑 Skip if blob already exists
    if container_client.get_blob_client(blob_chunk_name).exists():
        print(f"⏭️ Chunk {chunk_count} already exists in Azure. Skipping...")
        continue

    print(f"🚧 Processing chunk {chunk_count}")

    # Fetch a chunk of documents
    cursor = collection.find().skip(skip).limit(CHUNK_SIZE)
    chunk_data = list(cursor)

    # Convert to DataFrame
    df_chunk = pd.DataFrame.from_records(chunk_data)

    # Save to CSV
    local_chunk_file_name = f"mongo_collection_data_chunk_{chunk_count}.csv"
    df_chunk.to_csv(local_chunk_file_name, index=False)

    # Upload to Azure
    with open(local_chunk_file_name, "rb") as data:
        container_client.upload_blob(name=blob_chunk_name, data=data, overwrite=True)

    print(f"✅ Chunk {chunk_count} uploaded as '{blob_chunk_name}'")


Total documents in collection: 2204490
⏭️ Chunk 1 already exists in Azure. Skipping...
⏭️ Chunk 2 already exists in Azure. Skipping...
⏭️ Chunk 3 already exists in Azure. Skipping...
⏭️ Chunk 4 already exists in Azure. Skipping...
🚧 Processing chunk 5
✅ Chunk 5 uploaded as '_chunk_5.csv'
